In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summarization-using-pegasus/data.csv
/kaggle/input/summarization-using-pegasus-jayant/Summarization_dataset.csv
/kaggle/input/summarization-final-dataset/final_dataset.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tweet-preprocessor
!pip install rouge-score

In [ ]:
import sys
import io
import pandas as pd
import numpy as np
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AdamW
import torch
from datasets import load_dataset, Dataset, load_metric, DatasetDict
from transformers.optimization import Adafactor, AdafactorSchedule
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
# import evaluate
import gc
import re
from torch import nn
import preprocessor as p


torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## LOAD THE DATASET

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/NLP_project/data/final_dataset.csv')
dataset

,inputs,summaries
0,Artificial intelligence is transforming the he...,AI is revolutionizing healthcare by improving ...
1,The future of transportation is electric. With...,"EVs are the future of transportation, and as w..."
2,"Remote work is here to stay, and it's changing...",Remote work is changing the way we balance wor...
3,Blockchain technology is transforming the way ...,Blockchain is transforming business by enablin...
4,The global food system is facing unprecedented...,Building a sustainable and equitable food syst...
...,...,...
1346,"""It's important to make time for the people an...",The tweet is about prioritizing the things and...
1347,"""Just took my dog for a long walk and feeling ...",The tweet is about going for a walk with a pet...
1348,"""Happiness is not a destination, but a journey...",The tweet is about the importance of enjoying ...
1349,"""Excited to announce that I'll be starting a n...",The tweet is about starting a new job and expr...


## PREPROCESSING

Tweets usually contain hashtags, emojis, links etc. So, before applying our model, we have to clean the dataset. To remove these unwanted characters, I have used tweet-preprocessor library which is twitter specific preprocessing library.

In [ ]:
for i in range(58,64):
    print(dataset['inputs'][i])

Nothing beats a morning run to start the day off right! 🏃‍♂️☀️ #morningmotivation
Attended a yoga class for the first time today and it was such a calming experience 🧘‍♀️💆‍♀️ #yogalife
Just finished reading a book on mindfulness and it has changed my perspective on life 📚🙏 #mindfulness
Started a new workout routine and already feeling the burn 🔥💪 #workoutmotivation
Took a dance class for the first time in years and it was so much fun! 💃🎉 #dancelife
Just finished a challenging hike and the view at the top was worth it! 🌄🥾 #hikingadventures


In [ ]:

for i in range(len(dataset['inputs'])):
    dataset['inputs'][i] = p.clean(dataset['inputs'][i])
    dataset['summaries'][i] = p.clean(dataset['summaries'][i])

In [ ]:
for i in range(58,64):
    print(dataset['inputs'][i])

Nothing beats a morning run to start the day off right!
Attended a yoga class for the first time today and it was such a calming experience
Just finished reading a book on mindfulness and it has changed my perspective on life
Started a new workout routine and already feeling the burn
Took a dance class for the first time in years and it was so much fun!
Just finished a challenging hike and the view at the top was worth it!


### SOUNDEX

I have used soundex to handle lexical variations in tweets. Often, the words we use while writing a tweet/message vary from person to person.
Following can be the lexical variations of the word "tomorrow":
‘2marrow’,  ‘2morow’, ‘2morrow’,
‘tmmrw’, , ‘tmorrow’ ‘tmrow’, ‘tmrrow’, ‘tmrrw’, ‘tmrw’, ‘tmrww’, ‘tmw’,
Thus, we need to normalize the words.
I tried soundex to normalize them.

In [ ]:
# import unicodedata
# def soundex(s):

#     if not s:
#         return ""

#     s = unicodedata.normalize("NFKD", s)
#     s = s.upper()

#     replacements = (
#         ("BFPV", "1"),
#         ("CGJKQSXZ", "2"),
#         ("DT", "3"),
#         ("L", "4"),
#         ("MN", "5"),
#         ("R", "6"),
#     )
#     result = [s[0]]
#     count = 1

#     # find would-be replacment for first character
#     for lset, sub in replacements:
#         if s[0] in lset:
#             last = sub
#             break
#     else:
#         last = None

#     for letter in s[1:]:
#         for lset, sub in replacements:
#             if letter in lset:
#                 if sub != last:
#                     result.append(sub)
#                     count += 1
#                 last = sub
#                 break
#         else:
#             if letter != "H" and letter != "W":
#                 # leave last alone if middle letter is H or W
#                 last = None
#         if count == 4:
#             break

#     result += "0" * (4 - count)
#     return "".join(result)

In [ ]:
# for i in range(len(dataset['inputs'])):
#     dataset['inputs'][i] = p.clean(dataset['inputs'][i])
#     dataset['summaries'][i] = p.clean(dataset['summaries'][i])
#     temp = dataset['inputs'][i]
#     temp = temp.split(" ")
#     updated_input=""
#     for word in temp:
#         updated_input+=' '+ soundex(word)
#     dataset['inputs'][i] = updated_input
#     temp = dataset['summaries'][i]
#     updated_summaries = ""
#     for word in temp:
#         updated_summaries+=' '+ soundex(word)
#     dataset['summaries'][i] = updated_summaries

### Splitting into train:validation:test :: 80:10:10

In [ ]:
train,rem = train_test_split(dataset, train_size=0.8)
valid,test = train_test_split(rem, test_size=0.5)

## Converting the dataset into model specific datatype

In [ ]:
train = Dataset.from_dict(train)
test = Dataset.from_dict(test)
valid = Dataset.from_dict(valid)

In [ ]:
tweets = DatasetDict({'train':train, 'valid':valid, 'test':test})
tweets

DatasetDict({
    train: Dataset({
        features: ['inputs', 'summaries'],
        num_rows: 1080
    })
    valid: Dataset({
        features: ['inputs', 'summaries'],
        num_rows: 135
    })
    test: Dataset({
        features: ['inputs', 'summaries'],
        num_rows: 136
    })
})

### LOADING THE MODEL(pegasus-xsum), TOKENIZER(pegasus), METRIC(rogue)

In [ ]:
model_name = 'google/pegasus-xsum'

# INITIALIZING PEGASUS TOKENIZER
tokenizer = PegasusTokenizer.from_pretrained(model_name)

#INITIALIZING PEGASUS MODEL
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# EVALUATION METRIC
metric = load_metric("rouge")

## FREEZING THE EMBEDDINGS


To save the computation time in the Transformer package,
embeddings are pre-computed up to the length of 512 and
stored as a variable that serves as a cache which should not change during training.

The reason for not training the position embeddings is that the embeddings for later positions would be undertrained,
 but with cleverly analytically defined position embeddings,
 the network can learn the regularity behind the equations and generalize for longer sequences more easily.

In [ ]:

def freeze_params(model):
    for param in model.parameters():
        param.requires_grad = False

freeze_params(model.model.shared)
for d in [model.model.encoder, model.model.decoder]:
  freeze_params(d.embed_positions)
  freeze_params(d.embed_tokens)

# # Freeze the embeddings
# model.model.encoder.requires_grad = False
# model.model.decoder.requires_grad = False

## TOKENIZE THE INPUTS AND SUMMARIES

Before we can feed those texts to our model, we need to preprocess them. This is done by a pegasus tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

In [ ]:
max_input_length = tokenizer.max_model_input_sizes[model_name]
# MAX_MODEL_INPUT_SIZE FOR PEGASUS IS 512
max_summary_length = 64

def tokenizer_function(curr_dataset):
    # Tokenizer for inputs
    inputs = tokenizer(curr_dataset["inputs"], max_length=max_input_length, truncation=True)

    # Tokenizer for summaries
    summaries = tokenizer(curr_dataset["summaries"], max_length=max_summary_length, truncation=True)

    ## Merging the tokens of summaries and inputs
    inputs["labels"] = summaries["input_ids"]
    # print(inputs)
    return inputs

tokenized_tweets = tweets.map(tokenizer_function,batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## INITIALIZING ARGUMENTS


### ARGUMENTS
* evaluation_strategy - The evaluation strategy to adopt during training. "epoch" means evaluation is done at the end of each epoch.
* per_device_train/eval_batch_size - The batch size per GPU/TPU core/CPU for training/eval.
* weight_decay - The weight decay to apply to all layers except all bias and LayerNorm weights in AdamW optimizer.
* predict_with_generate — Whether to use generate to calculate generative metrics.
* save_total_limit -  If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir.


In [ ]:
args = Seq2SeqTrainingArguments(
    evaluation_strategy = "epoch", # evaluation is done at the end of each epoch.
    learning_rate=5e-5,
    per_device_train_batch_size=1, # batch size per device during training
    per_device_eval_batch_size=1,  # batch size for evaluation
    weight_decay=0.005,            # strength of weight decay
    warmup_steps=500,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    output_dir = 'none',
)


# Data collators are objects that will form a batch by using a list of dataset elements as input.
# These elements are of the same type as the elements of train_dataset or eval_dataset.

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Optimizer : Parameters of the optimizer are taken as recommended in the documentation.
optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True)
lr_scheduler = AdafactorSchedule(optimizer)


I didn't apply weight decay to 'bias' & 'LayerNorm' layers following the standards. But the ROUGE score was less. Below is the code for the approach.

In [ ]:
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# params_0 = [p for n,p in named_parameters if "embeddings" in n
#                 and any(nd in n for nd in no_decay)]
# params_1 = [p for n,p in named_parameters if "embeddings" in n
#                 and not any(nd in n for nd in no_decay)]

# embed_params = {"params": params_0, "lr": lr, "weight_decay": 0.0}
# opt_parameters.append(embed_params)
#embed_params = {"params": params_1, "lr": lr, "weight_decay": 0.01}
#opt_parameters.append(embed_params)
# optimizer = Adafactor(opt_parameters, scale_parameter=True, relative_step=True, warmup_init=True)
# lr_scheduler = AdafactorSchedule(optimizer)


## FUNCTION TO COMPUTE ROGUE SCORE INSIDE THE TRAINING LOOP

In [ ]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    print(result)
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

## TRAINING THE MODEL WITH APPROPRIATE PARAMETERS

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_tweets["train"],
    eval_dataset=tokenized_tweets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=metric_fn
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.563500,1.074569,64.337232,46.307959,58.528214,58.535784,26.540741
2,1.012000,1.091732,59.396322,41.513633,54.007965,54.045972,21.829630
3,0.692200,1.226390,62.350464,43.535124,56.860943,57.119873,24.992593
4,0.575200,1.413349,62.216377,43.183369,56.427634,56.550777,24.822222


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge1': 64.33723201608903, 'rouge2': 46.307958960807774, 'rougeL': 58.52821400839234, 'rougeLsum': 58.53578430984419}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge1': 62.35046437502794, 'rouge2': 43.53512357827084, 'rougeL': 56.86094266999548, 'rougeLsum': 57.11987330563038}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'rouge1': 62.216377293792156, 'rouge2': 43.18336889816483, 'rougeL': 56.42763440459995, 'rougeLsum': 56.550776742116696}


TrainOutput(global_step=2160, training_loss=0.9348672548929851, metrics={'train_runtime': 2708.9485, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.797, 'total_flos': 509426120589312.0, 'train_loss': 0.9348672548929851, 'epoch': 4.0})

## GENERATING SUMMARIES OF TEST TWEETS

In [ ]:
output = trainer.predict(tokenized_tweets["test"])
sz = len(tokenized_tweets['test'])
generated_summaries = []
for i in range(0, sz):
  generated_summaries.append(tokenizer.decode(output[0][i], skip_special_tokens =  True))
given_summaries = tokenized_tweets["test"]["summaries"]
input_tweets = tokenized_tweets["test"]["inputs"]

{'rouge1': 61.87638352369107, 'rouge2': 43.52033446338909, 'rougeL': 56.54791548153695, 'rougeLsum': 56.614247582772805}


## METRICS OF TEST DATA

In [ ]:
output.metrics

{'test_loss': 1.3670787811279297,
 'test_rouge1': 61.87638352369107,
 'test_rouge2': 43.52033446338909,
 'test_rougeL': 56.54791548153695,
 'test_rougeLsum': 56.614247582772805,
 'test_gen_len': 23.095588235294116,
 'test_runtime': 89.6402,
 'test_samples_per_second': 1.517,
 'test_steps_per_second': 0.759}

## PRECISION, RECALL AND F1 SCORE

In [ ]:
!pip install bert_score
from bert_score import score
P, R, F1 = score(generated_summaries, given_summaries, lang="en", verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/5 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 1.53 seconds, 89.02 sentences/sec


In [ ]:
print(f"Recall score: {R.mean():.3f}")
print(f"Precision: {P.mean():.3f}")
print(f"F1 score: {F1.mean():.3f}")

Recall score: 0.937
Precision: 0.945
F1 score: 0.941


## HERE ARE 5 TWEETS, THEIR GIVEN SUMMARY AND SUMMARY GENERATED BY THE MODEL

### TWEETS

In [ ]:
input_tweets[0:5]

['India Against Corruption is a grassroots movement aimed at combating corruption in Indian society and promoting transparency and accountability in government.',
 'Twitter is constantly evolving and changing, with new features and updates being rolled out regularly. Let us stay informed and adapt to these changes to make the most of this platform.',
 'Apple has always been at the forefront of innovation, and with the recent launch of the iPhone , they have once again proven why they are the leading technology company in the world. The new device boasts improved cameras, a faster processor, and a longer battery life, making it a must-have for anyone looking for the latest and greatest smartphone.',
 "Tesla's Model S Plaid is the fastest production car ever made, with mind-boggling acceleration, top speed, and performance that puts it ahead of the competition.",
 "Microsoft's cloud computing platform, Azure, is one of the fastest-growing cloud services in the world. It offers a wide ran

### GIVEN SUMMARIES

In [ ]:
given_summaries[0:5]

['This tweet introduces the India Against Corruption movement, highlighting its goal of eradicating corruption in Indian society and promoting government transparency and accountability.',
 "A recognition of Twitter's constantly evolving nature and a call to stay informed and adapt to changes.",
 "Apple's iPhone launch shows their commitment to innovation with improved cameras, faster processors and a longer battery life.",
 "Tesla's Model S Plaid is the fastest production car ever made, with unmatched acceleration, top speed, and performance.",
 'Azure is a rapidly-growing cloud computing platform offered by Microsoft, with a variety of services including machine learning and data storage.']

### GENERATED SUMMARIES

In [ ]:
generated_summaries[0:5]

["This tweet notes the grassroots movement's aim to combat corruption in Indian society and promote transparency and accountability in government.",
 'A call to stay informed and adapt to the latest features and updates on Twitter.',
 "Apple's iPhone offers improved cameras, faster processor, and longer battery life.",
 "Tesla's Model S Plaid is the fastest production car ever, with incredible acceleration, top speed, and performance.",
 "Microsoft's cloud computing platform, Azure, offers a wide range of services, including virtual machines, data storage, and machine learning."]